<a href="https://colab.research.google.com/github/SamyukthaYerra/Brainwave_Matrix_Intern/blob/main/twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing kaggle library
! pip install kaggle

In [ ]:
#configuring the path of kaggle.json file
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
#API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#extracting the compressed dataset

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words('english'))


['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
twitter_dataset = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='latin-1')

In [ ]:
twitter_dataset.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
column_names = ['target','ids','date','flag','user','text']
twitter_dataset.columns = column_names

In [ ]:
twitter_dataset.head()

,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
twitter_dataset.shape

(1599999, 6)

In [ ]:
twitter_dataset.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
twitter_dataset['target'].value_counts()

,count
target,
4,800000
0,799999


In [ ]:
twitter_dataset.replace({'target':{4:1}},inplace=True)

In [ ]:
twitter_dataset['target'].value_counts()

,count
target,
1,800000
0,799999


In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):

  stemming = re.sub('[^a-zA-Z]',' ',content)
  stemming = stemming.lower()
  stemming = stemming.split()
  stemming = [port_stem.stem(word) for word in stemming if not word in stopwords.words('english')]
  stemming = ' '.join(stemming)
  return stemming


In [ ]:
twitter_dataset['stemming'] = twitter_dataset['text'].apply(stemming)

In [ ]:
twitter_dataset.head()

,target,ids,date,flag,user,text,stemming
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew,kwesidei whole crew


In [ ]:
print(twitter_dataset['stemming'])

0          upset updat facebook text might cri result sch...
1          kenichan dive mani time ball manag save rest g...
2                            whole bodi feel itchi like fire
3                              nationwideclass behav mad see
4                                        kwesidei whole crew
                                 ...                        
1599994                           woke school best feel ever
1599995    thewdb com cool hear old walt interview http b...
1599996                         readi mojo makeov ask detail
1599997    happi th birthday boo alll time tupac amaru sh...
1599998    happi charitytuesday thenspcc sparkschar speak...
Name: stemming, Length: 1599999, dtype: object


In [ ]:
print(twitter_dataset['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599994    1
1599995    1
1599996    1
1599997    1
1599998    1
Name: target, Length: 1599999, dtype: int64


In [ ]:
X = twitter_dataset['stemming'].values
Y = twitter_dataset['target'].values

In [ ]:
print(X)

['upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound'
 'whole bodi feel itchi like fire' ... 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

(1599999,) (1279999,) (320000,)


In [ ]:
print(X_train)

['watch saw iv drink lil wine'
 'hire anoth employe gourmet point current hurri'
 'punish know much work tomorrow everyon els get day' ...
 'ohjustjak awkward crush nph amazingg'
 'oliveandfig wow never tweet nah yet tuesday spoil'
 'girl jenn stay twitter b c got back late amp tri relax enough sleep work amp awak']


In [ ]:
print(X_test)

['school someon slap acrosss face caus stay class total dragggg poor poor kitti'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'dad comput turn even though weird mean download song want' ...
 'destini nevertheless hooray member wonder safe trip'
 'strawberri heavi appar broke blender epic fuck fail' 'supersandro thank']


In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)

X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2289342 stored elements and shape (320000, 461471)>
  Coords	Values
  (0, 63768)	0.26422682589785035
  (0, 75261)	0.26552138110472423
  (0, 127038)	0.2730995033465573
  (0, 219133)	0.3295455088887744
  (0, 320624)	0.5238287544743887
  (0, 355868)	0.22363572756759686
  (0, 371332)	0.3972681545090477
  (0, 375613)	0.24658637821166524
  (0, 382367)	0.2541115193458825
  (0, 414033)	0.2624690535388268
  (1, 6452)	0.3061987253241848
  (1, 15098)	0.21044041574811612
  (1, 145408)	0.5774478727480551
  (1, 217431)	0.40260807391422926
  (1, 256686)	0.2861349119561766
  (1, 348128)	0.4737202797635419
  (1, 366259)	0.24522188688615762
  (2, 79341)	0.34512546063620464
  (2, 88422)	0.33529803711277506
  (2, 107866)	0.36857211925047617
  (2, 124443)	0.2729825984376057
  (2, 258609)	0.3072614985829275
  (2, 375890)	0.3020734075258265
  (2, 407333)	0.27060796399382786
  (2, 418804)	0.33257344872506
  :	:
  (319995, 193560)	0.373116119782695


In [ ]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453002 stored elements and shape (1279999, 461471)>
  Coords	Values
  (0, 109298)	0.37470892870977707
  (0, 185125)	0.5296934860731303
  (0, 234920)	0.4196620311217523
  (0, 354566)	0.358082129508793
  (0, 436655)	0.27247657460069113
  (0, 443035)	0.44770025018080273
  (1, 19762)	0.25286013616648123
  (1, 86730)	0.33015139495167095
  (1, 119689)	0.4309123985667578
  (1, 152015)	0.49196931362728563
  (1, 166826)	0.3977368489124923
  (1, 172343)	0.3785321144028949
  (1, 319721)	0.3128279791558143
  (2, 93826)	0.2233049434503089
  (2, 117914)	0.3986115507964782
  (2, 124728)	0.33175938915396974
  (2, 146003)	0.2213048673720909
  (2, 220710)	0.2626149875630723
  (2, 279017)	0.2849974993622606
  (2, 326288)	0.5853283798404599
  (2, 412554)	0.28961919890983556
  (2, 445635)	0.23563700521896036
  (3, 46285)	0.6887696820070169
  (3, 93826)	0.46202690964703447
  (3, 331805)	0.4656145237310222
  :	:
  (1279996, 85239)	0.3425039810672

In [ ]:
model=LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [ ]:
print('Accuracy score of the training data : ',training_data_accuracy)

Accuracy score of the training data :  0.801063125830567


In [ ]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [ ]:
print('Accuracy score of the test data : ',test_data_accuracy)

Accuracy score of the test data :  0.7771625


In [ ]:
import pickle

In [ ]:
filename = 'twitter_sentiment_analysis_model.sav'
pickle.dump(model,open(filename,'wb'))

In [ ]:
loaded_model = pickle.load(open('/content/twitter_sentiment_analysis_model.sav','rb'))

In [ ]:
X_new = X_test[200]
print(Y_test[200])

prediction = model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

1
[1]
Positive Tweet


In [ ]:
X_new = X_test[3]
print(Y_test[3])

prediction = model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

0
[1]
Positive Tweet


In [ ]:
#Step 1: Train the model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib

model = LogisticRegression()
model.fit(X_train, Y_train)

# Step 2: Evaluate the model
Y_pred = model.predict(X_test)

print("✅ Model Evaluation")
print("Accuracy:", accuracy_score(Y_test, Y_pred))
print("Classification Report:\n")
print(classification_report(Y_test, Y_pred, target_names=["Negative", "Positive"]))

# Step 3: Test with one sample from test set
X_new = X_test[3]
print("\n✅ Testing a sample tweet from test set:")
print("Actual Label:", Y_test[3])
prediction = model.predict([X_new])
print("Predicted Label:", prediction[0])
print("Predicted Sentiment:", "Positive Tweet" if prediction[0] == 1 else "Negative Tweet")





✅ Model Evaluation
Accuracy: 0.7771625
Classification Report:

              precision    recall  f1-score   support

    Negative       0.79      0.76      0.77    160000
    Positive       0.77      0.80      0.78    160000

    accuracy                           0.78    320000
   macro avg       0.78      0.78      0.78    320000
weighted avg       0.78      0.78      0.78    320000


✅ Testing a sample tweet from test set:
Actual Label: 0


ValueError: Expected 2D array, got 1D array instead:
array=[<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 9 stored elements and shape (1, 461471)>         ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.